# Heartattack prediction

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer,SimpleImputer

from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, multilabel_confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.datasets import load_wine

In [2]:
df = pd.read_csv('heart.csv')
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


# preprocessing

In [3]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
counter = 0
for i in df.columns:
    print(f'{df.columns[counter]}:\n{df[i].unique()}')
    counter +=1

age:
[63 37 41 56 57 44 52 54 48 49 64 58 50 66 43 69 59 42 61 40 71 51 65 53
 46 45 39 47 62 34 35 29 55 60 67 68 74 76 70 38 77]
sex:
[1 0]
cp:
[3 2 1 0]
trestbps:
[145 130 120 140 172 150 110 135 160 105 125 142 155 104 138 128 108 134
 122 115 118 100 124  94 112 102 152 101 132 148 178 129 180 136 126 106
 156 170 146 117 200 165 174 192 144 123 154 114 164]
chol:
[233 250 204 236 354 192 294 263 199 168 239 275 266 211 283 219 340 226
 247 234 243 302 212 175 417 197 198 177 273 213 304 232 269 360 308 245
 208 264 321 325 235 257 216 256 231 141 252 201 222 260 182 303 265 309
 186 203 183 220 209 258 227 261 221 205 240 318 298 564 277 214 248 255
 207 223 288 160 394 315 246 244 270 195 196 254 126 313 262 215 193 271
 268 267 210 295 306 178 242 180 228 149 278 253 342 157 286 229 284 224
 206 167 230 335 276 353 225 330 290 172 305 188 282 185 326 274 164 307
 249 341 407 217 174 281 289 322 299 300 293 184 409 259 200 327 237 218
 319 166 311 169 187 176 241 131]
fbs:
[1 0]

### By observing we can say that there is no null value or any special character is present in columns

In [6]:
df['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

In [7]:
df.corr()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
age,1.000000,-0.098447,-0.068653,0.279351,0.213678,0.121308,-0.116211,-0.398522,0.096801,0.210013,-0.168814,0.276326,0.068001,-0.225439
sex,-0.098447,1.000000,-0.049353,-0.056769,-0.197912,0.045032,-0.058196,-0.044020,0.141664,0.096093,-0.030711,0.118261,0.210041,-0.280937
cp,-0.068653,-0.049353,1.000000,0.047608,-0.076904,0.094444,0.044421,0.295762,-0.394280,-0.149230,0.119717,-0.181053,-0.161736,0.433798
trestbps,0.279351,-0.056769,0.047608,1.000000,0.123174,0.177531,-0.114103,-0.046698,0.067616,0.193216,-0.121475,0.101389,0.062210,-0.144931
chol,0.213678,-0.197912,-0.076904,0.123174,1.000000,0.013294,-0.151040,-0.009940,0.067023,0.053952,-0.004038,0.070511,0.098803,-0.085239
fbs,0.121308,0.045032,0.094444,0.177531,0.013294,1.000000,-0.084189,-0.008567,0.025665,0.005747,-0.059894,0.137979,-0.032019,-0.028046
restecg,-0.116211,-0.058196,0.044421,-0.114103,-0.151040,-0.084189,1.000000,0.044123,-0.070733,-0.058770,0.093045,-0.072042,-0.011981,0.137230
thalach,-0.398522,-0.044020,0.295762,-0.046698,-0.009940,-0.008567,0.044123,1.000000,-0.378812,-0.344187,0.386784,-0.213177,-0.096439,0.421741
exang,0.096801,0.141664,-0.394280,0.067616,0.067023,0.025665,-0.070733,-0.378812,1.000000,0.288223,-0.257748,0.115739,0.206754,-0.436757
oldpeak,0.210013,0.096093,-0.149230,0.193216,0.053952,0.005747,-0.058770,-0.344187,0.288223,1.000000,-0.577537,0.222682,0.210244,-0.430696


In [8]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


# scaling

In [11]:
new_df = df.copy()
x = new_df.drop('target', axis = 1)
y = new_df['target']

In [12]:
std_scaler = StandardScaler()
std_scaler.fit(x)  # Saving mean and std

StandardScaler()

In [13]:
standard_array = std_scaler.transform(x)
x_df_std = pd.DataFrame(standard_array, columns = x.columns)
x_df_std

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.952197,0.681005,1.973123,0.763956,-0.256334,2.394438,-1.005832,0.015443,-0.696631,1.087338,-2.274579,-0.714429,-2.148873
1,-1.915313,0.681005,1.002577,-0.092738,0.072199,-0.417635,0.898962,1.633471,-0.696631,2.122573,-2.274579,-0.714429,-0.512922
2,-1.474158,-1.468418,0.032031,-0.092738,-0.816773,-0.417635,-1.005832,0.977514,-0.696631,0.310912,0.976352,-0.714429,-0.512922
3,0.180175,0.681005,0.032031,-0.663867,-0.198357,-0.417635,0.898962,1.239897,-0.696631,-0.206705,0.976352,-0.714429,-0.512922
4,0.290464,-1.468418,-0.938515,-0.663867,2.082050,-0.417635,0.898962,0.583939,1.435481,-0.379244,0.976352,-0.714429,-0.512922
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.290464,-1.468418,-0.938515,0.478391,-0.101730,-0.417635,0.898962,-1.165281,1.435481,-0.724323,-0.649113,-0.714429,1.123029
299,-1.033002,0.681005,1.973123,-1.234996,0.342756,-0.417635,0.898962,-0.771706,-0.696631,0.138373,-0.649113,-0.714429,1.123029
300,1.503641,0.681005,-0.938515,0.706843,-1.029353,2.394438,0.898962,-0.378132,-0.696631,2.036303,-0.649113,1.244593,1.123029
301,0.290464,0.681005,-0.938515,-0.092738,-2.227533,-0.417635,0.898962,-1.515125,1.435481,0.138373,-0.649113,0.265082,1.123029


In [ ]:
Need to use SMOTE for balancing the data

In [14]:
from imblearn.over_sampling import SMOTE

In [15]:
smt = SMOTE()
x_samples, y_samples = smt.fit_resample(x_df_std,y)
y_samples.value_counts()

1    165
0    165
Name: target, dtype: int64

In [16]:
x_samples

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.952197,0.681005,1.973123,0.763956,-0.256334,2.394438,-1.005832,0.015443,-0.696631,1.087338,-2.274579,-0.714429,-2.148873
1,-1.915313,0.681005,1.002577,-0.092738,0.072199,-0.417635,0.898962,1.633471,-0.696631,2.122573,-2.274579,-0.714429,-0.512922
2,-1.474158,-1.468418,0.032031,-0.092738,-0.816773,-0.417635,-1.005832,0.977514,-0.696631,0.310912,0.976352,-0.714429,-0.512922
3,0.180175,0.681005,0.032031,-0.663867,-0.198357,-0.417635,0.898962,1.239897,-0.696631,-0.206705,0.976352,-0.714429,-0.512922
4,0.290464,-1.468418,-0.938515,-0.663867,2.082050,-0.417635,0.898962,0.583939,1.435481,-0.379244,0.976352,-0.714429,-0.512922
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,1.574466,0.681005,-0.938515,0.101134,0.982559,-0.417635,-1.005832,-1.549892,-0.696631,1.095515,-0.649113,2.002438,-0.883142
326,-0.797957,0.681005,-0.938515,-0.609089,0.212795,-0.417635,-1.005832,0.819615,-0.696631,-0.559205,-0.296053,-0.501673,0.767691
327,-1.082975,0.681005,1.973123,-1.198027,0.301476,-0.417635,0.898962,-0.630174,-0.558620,0.283562,-0.649113,-0.714429,1.123029
328,-1.313127,0.681005,-0.938515,-0.721631,-0.240315,-0.417635,-0.042572,-0.335399,-0.696631,-0.032198,0.154350,-0.714429,1.123029


# train test split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_samples, y_samples, test_size = 0.45, random_state=11, stratify=y_samples)
x_train

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
25,1.834507,-1.468418,0.032031,1.620649,1.077125,-0.417635,0.898962,0.540209,-0.696631,-0.551783,0.976352,1.244593,-0.512922
230,-0.812425,0.681005,1.002577,-1.349222,-0.063079,-0.417635,0.898962,0.102904,-0.696631,-0.896862,0.976352,-0.714429,-0.512922
167,0.841908,-1.468418,-0.938515,0.478391,0.420058,-0.417635,-1.005832,0.452748,-0.696631,2.208842,-2.274579,1.244593,-0.512922
85,1.393352,-1.468418,1.002577,-0.949431,6.140401,-0.417635,-1.005832,0.452748,-0.696631,0.483451,-0.649113,-0.714429,1.123029
11,-0.702136,-1.468418,1.002577,-0.092738,0.555337,-0.417635,0.898962,-0.465593,-0.696631,-0.724323,0.976352,-0.714429,-0.512922
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,-0.371269,-1.468418,1.002577,0.478391,1.193077,-0.417635,-1.005832,-0.334401,-0.696631,0.397182,0.976352,0.265082,-0.512922
17,1.283063,-1.468418,1.973123,1.049520,-0.391612,-0.417635,0.898962,-1.558855,-0.696631,1.346147,-2.274579,-0.714429,-0.512922
81,-1.033002,0.681005,0.032031,-0.206964,1.193077,-0.417635,-1.005832,0.890053,-0.696631,-0.896862,0.976352,-0.714429,-0.512922
2,-1.474158,-1.468418,0.032031,-0.092738,-0.816773,-0.417635,-1.005832,0.977514,-0.696631,0.310912,0.976352,-0.714429,-0.512922


In [18]:
y_test.value_counts()

1    75
0    74
Name: target, dtype: int64

# Model training

In [19]:
log_reg = LogisticRegression(random_state=10)
log_reg.fit(x_train,y_train)

LogisticRegression(random_state=10)

# Evaluation

In [20]:
## Testing daat evaluation
y_pred_test = log_reg.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred_test)
print('Confusion matrix : \n', cnf_matrix)

multilabel_cnf_matrix = multilabel_confusion_matrix(y_test, y_pred_test)
print('Multilabel Confusion matrix : \n', multilabel_cnf_matrix)

acc_score_Logistic_Test = accuracy_score(y_test, y_pred_test)
print('Accuracy score : \n', acc_score_Logistic_Test)

class_report = classification_report(y_test, y_pred_test)
print('Classification report : \n', class_report)


Confusion matrix : 
 [[60 14]
 [12 63]]
Multilabel Confusion matrix : 
 [[[63 12]
  [14 60]]

 [[60 14]
  [12 63]]]
Accuracy score : 
 0.825503355704698
Classification report : 
               precision    recall  f1-score   support

           0       0.83      0.81      0.82        74
           1       0.82      0.84      0.83        75

    accuracy                           0.83       149
   macro avg       0.83      0.83      0.83       149
weighted avg       0.83      0.83      0.83       149



In [21]:
# Training Data Evalutaion

y_pred_train = log_reg.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred_train)
print('Confusuion matrix :\n', cnf_matrix)

acc_score__Logistic_Train = accuracy_score(y_train,y_pred_train)
print('Accuracy score :\n', acc_score__Logistic_Train)

clf_report = classification_report(y_train,y_pred_train)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[75 16]
 [ 7 83]]
Accuracy score :
 0.8729281767955801
Classification report :
               precision    recall  f1-score   support

           0       0.91      0.82      0.87        91
           1       0.84      0.92      0.88        90

    accuracy                           0.87       181
   macro avg       0.88      0.87      0.87       181
weighted avg       0.88      0.87      0.87       181



# BY KNN

# Model training

In [22]:
knn_clf = KNeighborsClassifier(n_neighbors=5)  
knn_clf.fit(x_train,y_train)  # model saves training data

KNeighborsClassifier()

In [23]:
# Testing Data Evalutaion

y_pred = knn_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print('Confusuion matrix :\n', cnf_matrix)

acc_score_KNN_Test = accuracy_score(y_test,y_pred)
print('Accuracy score :\n', acc_score_KNN_Test)

clf_report = classification_report(y_test,y_pred)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[55 19]
 [11 64]]
Accuracy score :
 0.7986577181208053
Classification report :
               precision    recall  f1-score   support

           0       0.83      0.74      0.79        74
           1       0.77      0.85      0.81        75

    accuracy                           0.80       149
   macro avg       0.80      0.80      0.80       149
weighted avg       0.80      0.80      0.80       149



In [24]:
# Training Data Evalutaion

y_pred_train = knn_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred_train)
print('Confusuion matrix :\n', cnf_matrix)

acc_score_KNN_Train = accuracy_score(y_train,y_pred_train)
print('Accuracy score :\n', acc_score_KNN_Train)

clf_report = classification_report(y_train,y_pred_train)
print('Classification report :\n', clf_report)

Confusuion matrix :
 [[81 10]
 [ 6 84]]
Accuracy score :
 0.9116022099447514
Classification report :
               precision    recall  f1-score   support

           0       0.93      0.89      0.91        91
           1       0.89      0.93      0.91        90

    accuracy                           0.91       181
   macro avg       0.91      0.91      0.91       181
weighted avg       0.91      0.91      0.91       181



# Decision Tree classification

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x_samples, y_samples, test_size = 0.45, random_state=11, stratify=y_samples)

In [29]:
dt_clf = DecisionTreeClassifier(random_state=10)
dt_clf.fit(x_train,y_train)

DecisionTreeClassifier(random_state=10)

# Model Evaluation

In [30]:
# Testing Accuracy

y_pred = dt_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Test = accuracy_score(y_test, y_pred)
print("Accuracy Score : \n", acc_score_DT_Test)
print("********************************************")

clf_report = classification_report(y_test, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[51 23]
 [16 59]]
********************************************
Accuracy Score : 
 0.738255033557047
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.76      0.69      0.72        74
           1       0.72      0.79      0.75        75

    accuracy                           0.74       149
   macro avg       0.74      0.74      0.74       149
weighted avg       0.74      0.74      0.74       149

********************************************


In [31]:
# Training Accuracy

y_pred = dt_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Train = accuracy_score(y_train, y_pred)
print("Accuracy Score : \n", acc_score_DT_Train)
print("********************************************")

clf_report = classification_report(y_train, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[91  0]
 [ 0 90]]
********************************************
Accuracy Score : 
 1.0
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        91
           1       1.00      1.00      1.00        90

    accuracy                           1.00       181
   macro avg       1.00      1.00      1.00       181
weighted avg       1.00      1.00      1.00       181

********************************************


# Plot DT

In [32]:
plt.figure(figsize = (200,160))
plot_tree(dt_clf, feature_names=x.columns, class_names = ["0", "1"], filled = True)
plt.savefig('DecisionTree_Heart.png')

# Hyperparameter Testing

In [33]:
dt_clf = DecisionTreeClassifier()
hyperparameters = {"criterion" : ["gini", "entropy"],
                  "max_depth" : np.arange(3,8),
                  "min_samples_split" : np.arange(2,20),
                  "min_samples_leaf" : np.arange(2,10)}

In [34]:
rscv_dt_model = RandomizedSearchCV(dt_clf, hyperparameters, cv = 5)
rscv_dt_model.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([3, 4, 5, 6, 7]),
                                        'min_samples_leaf': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])})

In [35]:
rscv_dt_model.best_estimator_

DecisionTreeClassifier(max_depth=6, min_samples_leaf=3, min_samples_split=16)

In [36]:
dt_clf = DecisionTreeClassifier(max_depth=6, min_samples_leaf=3, min_samples_split=16)
dt_clf.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=3, min_samples_split=16)

In [37]:
# Testing Accuracy

y_pred = dt_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Test = accuracy_score(y_test, y_pred)
print("Accuracy Score : \n", acc_score_DT_Test)
print("********************************************")

clf_report = classification_report(y_test, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[56 18]
 [18 57]]
********************************************
Accuracy Score : 
 0.7583892617449665
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.76      0.76      0.76        74
           1       0.76      0.76      0.76        75

    accuracy                           0.76       149
   macro avg       0.76      0.76      0.76       149
weighted avg       0.76      0.76      0.76       149

********************************************


In [38]:
# Training Accuracy

y_pred = dt_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train, y_pred)
print("Confusion Matrix : \n", cnf_matrix)
print("********************************************")

acc_score_DT_Train = accuracy_score(y_train, y_pred)
print("Accuracy Score : \n", acc_score_DT_Train)
print("********************************************")

clf_report = classification_report(y_train, y_pred)
print("Classification report : \n", clf_report)
print("********************************************")

Confusion Matrix : 
 [[83  8]
 [ 6 84]]
********************************************
Accuracy Score : 
 0.9226519337016574
********************************************
Classification report : 
               precision    recall  f1-score   support

           0       0.93      0.91      0.92        91
           1       0.91      0.93      0.92        90

    accuracy                           0.92       181
   macro avg       0.92      0.92      0.92       181
weighted avg       0.92      0.92      0.92       181

********************************************


# Random Forest

In [39]:
rf_clf = RandomForestClassifier(random_state=10)
rf_clf.fit(x_train, y_train)

RandomForestClassifier(random_state=10)

In [40]:
# Testing Data Evaluation
y_pred = rf_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print("Confusion Matrix \n", cnf_matrix)

acc_score_RF_Test = accuracy_score(y_test,y_pred)
print("Accuracy score \n", acc_score_RF_Test)

clf_report = classification_report(y_test,y_pred)
print("Classification report \n", clf_report)

Confusion Matrix 
 [[56 18]
 [12 63]]
Accuracy score 
 0.7986577181208053
Classification report 
               precision    recall  f1-score   support

           0       0.82      0.76      0.79        74
           1       0.78      0.84      0.81        75

    accuracy                           0.80       149
   macro avg       0.80      0.80      0.80       149
weighted avg       0.80      0.80      0.80       149



In [41]:
# Training Data Evaluation
y_pred = rf_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred)
print("Confusion Matrix \n", cnf_matrix)

acc_score_RF_Train = accuracy_score(y_train,y_pred)
print("Accuracy score \n", acc_score_RF_Train)

clf_report = classification_report(y_train,y_pred)
print("Classification report \n", clf_report)

Confusion Matrix 
 [[91  0]
 [ 0 90]]
Accuracy score 
 1.0
Classification report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        91
           1       1.00      1.00      1.00        90

    accuracy                           1.00       181
   macro avg       1.00      1.00      1.00       181
weighted avg       1.00      1.00      1.00       181



# Hyperparameter Tuning

In [42]:
rf_model = RandomForestClassifier(random_state=10)

hyp = {"n_estimators" : np.arange(10,200),
    "criterion":['gini','entropy'],
    "max_depth":np.arange(5,15),
    "min_samples_split":np.arange(5,20),
    "min_samples_leaf":np.arange(5,15),
    "max_features":['auto']}

In [43]:
rscv_rf_clf =RandomizedSearchCV(rf_model, hyp, cv = 5)
rscv_rf_clf.fit(x_train,y_train)
rscv_rf_clf.best_estimator_

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=7, max_features='auto', min_samples_leaf=5,
                       min_samples_split=12, n_estimators=181, random_state=10)

In [45]:
rf_clf = RandomForestClassifier( max_depth=7, min_samples_leaf=5,max_features='auto'
                       ,min_samples_split=12, n_estimators=181, random_state=10)
rf_clf.fit(x_train,y_train)

C:\Users\Shruti\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=7, max_features='auto', min_samples_leaf=5,
                       min_samples_split=12, n_estimators=181, random_state=10)

In [52]:
# Testing Data Evaluation
y_pred = rf_clf.predict(x_test)

cnf_matrix = confusion_matrix(y_test,y_pred)
print("Confusion Matrix", cnf_matrix)

acc_score_RF_Test_hyper = accuracy_score(y_test,y_pred)
print("Accuracy score", acc_score_RF_Test_hyper)

clf_report = classification_report(y_test,y_pred)
print("Classification report", clf_report)

Confusion Matrix [[56 18]
 [15 60]]
Accuracy score 0.7785234899328859
Classification report               precision    recall  f1-score   support

           0       0.79      0.76      0.77        74
           1       0.77      0.80      0.78        75

    accuracy                           0.78       149
   macro avg       0.78      0.78      0.78       149
weighted avg       0.78      0.78      0.78       149



In [53]:
# Training Data Evaluation
y_pred = rf_clf.predict(x_train)

cnf_matrix = confusion_matrix(y_train,y_pred)
print("Confusion Matrix", cnf_matrix)

acc_score_RF_Train_hyper = accuracy_score(y_train,y_pred)
print("Accuracy score", acc_score_RF_Train_hyper)

clf_report = classification_report(y_train,y_pred)
print("Classification report", clf_report)

Confusion Matrix [[81 10]
 [ 3 87]]
Accuracy score 0.9281767955801105
Classification report               precision    recall  f1-score   support

           0       0.96      0.89      0.93        91
           1       0.90      0.97      0.93        90

    accuracy                           0.93       181
   macro avg       0.93      0.93      0.93       181
weighted avg       0.93      0.93      0.93       181



In [54]:
df_heart = pd.DataFrame(columns=['train_acc','Test_acc'],index=['Logistic_Heart','KNN_Heart','DT_Heart','RF_Heart'])
df_heart

,train_acc,Test_acc
Logistic_Heart,NaN,NaN
KNN_Heart,NaN,NaN
DT_Heart,NaN,NaN
RF_Heart,NaN,NaN


In [55]:
df_heart['train_acc'] = [acc_score__Logistic_Train,acc_score_KNN_Train,acc_score_DT_Train,acc_score_RF_Train_hyper]
df_heart['Test_acc'] = [acc_score_Logistic_Test,acc_score_KNN_Test,acc_score_DT_Test,acc_score_RF_Test_hyper]

In [56]:
df_heart.to_csv('df_heart.csv')
df_heart

,train_acc,Test_acc
Logistic_Heart,0.872928,0.825503
KNN_Heart,0.911602,0.798658
DT_Heart,0.922652,0.758389
RF_Heart,0.928177,0.778523
